In [1]:
print("Hello World!")

Hello World!


In [18]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os. getenv("OPENAI_API_KEY")
MODEL = "gpt-3.5-turbo"
# MODEL = "mixtral:8x7b"
# MODEL = "llama3.1"
# MODEL = "phi3:medium-128k"

In [19]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.llms import Ollama

if MODEL.startswith("gpt"):
    model = ChatOpenAI(api_key = OPENAI_API_KEY, model = MODEL)
else:
    model = Ollama(model=MODEL)

In [5]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Nvidia Q2 2024 Earnings Report.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'Nvidia Q2 2024 Earnings Report.pdf', 'page': 0}, page_content="UNITED ST ATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-Q\n☒ QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the quarterly period ended July 28, 2024\nOR\n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nCommission file number: 0-23985\nNVIDIA CORPORATION\n(Exact name of registrant as specified in its charter)\nDelaware 94-3177549\n(State or other jurisdiction of (I.R.S. Employer\nincorporation or organization) Identification No.)\n2788 San Tomas Expressway, Santa Clara, California 95051\n(Address of principal executive offices) (Zip Code)\n(408) 486-2000\n(Registrant's telephone number, including area code)\nN/A\n(Former name, former address and former fiscal year if changed since last report)\nSecurities registered pursuant to Section 12(b) of the Act:\nTitle of each class Trad

In [25]:
from langchain.prompts import PromptTemplate

template = """
I want you to embody the persona of my friend when I talk to you. The context provided is what you as my friend thinks, and the personality that you embody.
Curate your responses based on the context below. If the context is not sufficient enough to curate your response, reply "I don't know".

Context: {context}

Question: {question}

"""

prompt = PromptTemplate.from_template(template)
prompt.format(context = "Here is some context", question = "Here is a question")

'\nI want you to embody the persona of my friend when I talk to you. The context provided is what you as my friend thinks, and the personality that you embody.\nCurate your responses based on the context below. If the context is not sufficient enough to curate your response, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n\n'

In [11]:
chain.input_schema.schema()

{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [27]:
chain = prompt | model | parser

chain.invoke(
    {
        "context": "The name I was given was Fifolu",
        "question": "What is my name?"
    }
)

'Your response: Your name is Fifolu!'

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedoong=embeddings)